# 04_funcGLM1Loop

这是功能像的1st GLM。

In [ ]:
import os
from RZutilpy.system import Path, unix_wrapper, gettimestr, makedirs
from RZutilpy.rzio import matchfiles

# Step 1. 

首先，我们走到记录modulation的txt文件夹

In [ ]:
folders = matchfiles(f'/home/yufengxia/Data/23BPhappy/*_BPhappy_*')
#del folders[0:19] 
AFNI_HOME = os.getenv('AFNIDIR')
MNIT1 = AFNI_HOME + '/MNI152_2009_template_SSW.nii.gz'
Param1 = 'option_CRandEV'
Param2 = 'gamble_RPE'

for folderofsubj in folders:
    subj = folderofsubj[-4:]
    m_txt1 = Path(f'/home/yufengxia/Data/23BPhappy/parametric_modulation/{subj}_{Param1}_modulation.txt')
    m_txt2 = Path(f'/home/yufengxia/Data/23BPhappy/parametric_modulation/{subj}_{Param2}_modulation.txt')
    GLM_dir = Path(f'{folderofsubj}/funcGLM1/')
    GLM_dir = Path(GLM_dir)
    #assert not GLM_dir.exists(), f'GLM dir {GLM_dir} already exists'
    # verify that the results directory does not yet exist
    if not os.path.exists(GLM_dir):
        os.makedirs(GLM_dir)

    os.chdir(GLM_dir)   

    # ================================ regress =================================
    # compute de-meaned motion parameters (for use in regression)
    # in regress (detrend step)

    # ------------------------------
    # run the regression analysis. Note that stats results should be written as afni format
    #-x1D_stop 
    #-mask mask_epi_anat.{subj}.nii.gz \
    %cd {folderofsubj}/funcpp
    ! 3dDeconvolve -input pb07s.{subj}.r*.scale+tlrc.HEAD \
        -mask mask_epi_anat.{subj}.nii.gz \
        -censor motion_{subj}_censor.1D \
        -ortvec motion_demean.1D mot_demean \
        -polort 2 \
        -num_stimts 2 \
        -stim_times_AM2 1 {m_txt1} 'BLOCK5(1,1)' \
        -stim_times_AM2 2 {m_txt2} 'BLOCK5(1,1)' \
        -stim_label 1 {Param1} \
        -stim_label 2 {Param2} \
        -rout -fout -tout \
        -x1D $GLM_dir/{subj}_{Param1}_{Param2}_X1d \
        -xjpeg $GLM_dir/{subj}_{Param1}_{Param2}_X_output.jpg \
        -bucket $GLM_dir/{subj}_{Param1}_{Param2}_stats \
        -cbucket $GLM_dir/{subj}_{Param1}_{Param2}_cbucket \
        -errts $GLM_dir/{subj}_{Param1}_{Param2}_errts \
        -GOFORIT 4 




In [ ]:
#可视化结果，在ipython上定义好是哪个被试的funcGLM，然后用下面的代码一个个打开看
%cd {GLM_dir} 
! afni -dset {subj}_{Param1}_{Param2}_stats.nii.gz  {MNIT1}

In [ ]:
#如果出错，则删除funcGLM文件夹重来
folders = matchfiles(f'/home/yufengxia/Data/23BPhappy/*_BPhappy_*')
for folderofsubj in folders:
    subj = folderofsubj[-4:]
    GLM_dir = Path(f'{folderofsubj}/funcGLM1/')
    GLM_dir = Path(GLM_dir)
    # verify that the results directory does not yet exist
    ! rm -rf {GLM_dir}


In [ ]:
Group = scio.loadmat('/home/yufengxia/Data/23BPhappy/temp/clidata.mat')
Group = Group['clidata']



GroupMDD = []
GroupBD = []
#del folders[0:12] 

Param1 = 'option_CRandEV'
Param2 = 'gamble_CRandEV'
for i in range(0,len(Group)):
    subj = Group[i][0][0]
    folders = matchfiles(f'/home/yufengxia/Data/23BPhappy/*_BPhappy_{subj}')
    GLM_dir = Path(f'{folders}/funcGLM1')
    GLM_dir = Path(GLM_dir)
    if GLM_dir.exists() == False:
        continue

    if Group[i][10][0][0] == 0:
        GroupMDD.append(subj)
    elif Group[i][10][0][0] == 1:
            GroupBD.append(subj)

    statsfiles = matchfiles(f'{GLM_dir}/*{Param1}_{Param2}_stats.nii*')
    !cp {statsfiles}  /home/yufengxia/Data/23BPhappy/temp
    print(statsfiles)